---

<center><h1> GroupByKey Vs ReduceByKey </h1></center>

---

In [1]:
# importing the spark context
from pyspark.context import SparkContext

In [2]:
# spark context object
sc = SparkContext(appName="pair_rdd_operations")

---


We have a students data in the file **students_data.txt** which has the 9 different columns. First one is **Roll No**, **Section**, **Name of Student**, **City**, and the last five columns are marks in 5 different subjects. The data of each student is in different row separated by space.

---

![](images/data_1.png)


---


#### `Create the rdd of the file - students_data.txt`

---

In [3]:
## rdd of the file
students_marks = sc.textFile("data/students_data.txt")

In [4]:
## view the data
students_marks.collect()

['101 A Rohit Gurgaon 65 77 43 66 87',
 '102 B Akansha Delhi 55 46 24 66 77',
 '103 A Himanshu Faridabad 75 38 84 38 58',
 '104 A Ekta Delhi 85 84 39 58 85',
 '105 B Deepanshu Gurgaon 34 55 56 23 66',
 '106 B Ayush Delhi 66 62 98 74 87',
 '107 B Aditi Delhi 76 83 75 38 58',
 '108 A Sahil Faridabad 55 32 43 56 66',
 '109 A Krati Delhi 34 53 25 67 75']

---

Next, We will split each line into a list of words using Map. Let's see how to do this with the help of map transformation in the below cell. 

---

In [5]:
# split the rdd
students_marks = students_marks.map(lambda x: x.split(' '))

In [6]:
# collect the rdd
students_marks.collect()

[['101', 'A', 'Rohit', 'Gurgaon', '65', '77', '43', '66', '87'],
 ['102', 'B', 'Akansha', 'Delhi', '55', '46', '24', '66', '77'],
 ['103', 'A', 'Himanshu', 'Faridabad', '75', '38', '84', '38', '58'],
 ['104', 'A', 'Ekta', 'Delhi', '85', '84', '39', '58', '85'],
 ['105', 'B', 'Deepanshu', 'Gurgaon', '34', '55', '56', '23', '66'],
 ['106', 'B', 'Ayush', 'Delhi', '66', '62', '98', '74', '87'],
 ['107', 'B', 'Aditi', 'Delhi', '76', '83', '75', '38', '58'],
 ['108', 'A', 'Sahil', 'Faridabad', '55', '32', '43', '56', '66'],
 ['109', 'A', 'Krati', 'Delhi', '34', '53', '25', '67', '75']]

---

#### `Find out the number of students in each section?`


----

Now, we will create another paired RDD where key will be the section of the student and marks of students as the values.

----

In [7]:
# create pair rdd with section of students as key
students_section = students_marks.map(lambda x: (x[1], 1))

In [8]:
# collect the rdd
students_section.collect()

[('A', 1),
 ('B', 1),
 ('A', 1),
 ('A', 1),
 ('B', 1),
 ('B', 1),
 ('B', 1),
 ('A', 1),
 ('A', 1)]

---

#### `TRANSFORMATION - GROUPBYKEY`

---

It receives key-value pairs (K, V) as an input, group the values based on key and generates a dataset of (K, Iterable) pairs as an output.


---

![](images/group.png)

---

MapValues is applicable only for pair RDDs. As its name indicates, this transformation only operates on the values of the pair RDDs instead of operating on the whole tuple.

More about MapValues: 

 - http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=mapvalues#pyspark.RDD.mapValues

---

In [9]:
# group by key transformation
grouped_students_marks = students_section.groupByKey()

In [10]:
# collect the pair rdd
grouped_students_marks.mapValues(sum).collect()

[('A', 5), ('B', 4)]

----

#### `TRANSFORMATION -  REDUCEBYKEY`

---

It uses associative reduce function, where it merges value of each key. It can be used with Rdd only in key value pair.  It merges data locally using associative function for optimized data shuffling. 

---


![](images/reduce.png)


----

In [11]:
# reduce by key transformation
reduced_student_marks = students_section.reduceByKey(lambda x, y: x+y)

---
---

The below image shows how the reduce function works:

---

![](images/reduce.svg)

---

In [12]:
# collect the pair rdd
reduced_student_marks.collect()

[('A', 5), ('B', 4)]

---

#### `Comparison between GroupByKey and ReduceByKey`

---

We will create a sample data of 20 million data points and find out the sum of each key using both `groupByKey` and `reduceByKey`.

---

In [13]:
# importing the random library
import random

In [14]:
# empty list
my_section_list = []

# loop to create 20 million points
for i in range(20000000):
    my_section_list.append((random.choice(["A","B","C","D","E"]) , random.randint(1,10)))

---

Now, the above list will look something like this.

---

 [ ("A": 1),
 
   ("B": 2),
 
   ("C": 1),
   
   .
   
   .
   
   .
   
   ("A": 4)
]

----


**Create RDD of the collection**

----

In [15]:
# create rdd of the collection
rdd = sc.parallelize(my_section_list)

---

#### `GroupByKey`

---

In [16]:
# group the rdd
rdd_grouped = rdd.groupByKey()

In [17]:
# collect the results
rdd_grouped.mapValues(sum).collect()

[('B', 21994363),
 ('E', 22017465),
 ('D', 21988704),
 ('C', 21990282),
 ('A', 22029015)]

---

![](images/groupByKey.png)

----

---

In the above image, we can see that total 27 MB of data was shuffled across the partitions. Now, let's do the same task using the `reduceByKey` and see if there is any difference in the performance.

---

---

#### `ReduceByKey`

---

In [18]:
# reduce the rdd
rdd_reduced = rdd.reduceByKey(lambda x, y: x+y)

In [19]:
# collect the results
rdd_reduced.collect()

[('B', 21994363),
 ('E', 22017465),
 ('D', 21988704),
 ('C', 21990282),
 ('A', 22029015)]

---

![](images/reduceByKey.png)

---

---

Using `reduceByKey`, only 2.3 KB of data was shuffled. 

---